In [ ]:
from collections import defaultdict
import json
import ee
import pandas as pd
import geopandas as gpd
import geemap

In [ ]:
ee.Authenticate()

In [5]:
ee.Initialize()

In [ ]:
with open('popcodes.json', 'r') as popfile:
    popcodes2 = json.loads(popfile.read())

In [ ]:
ghspop_ic = ee.ImageCollection("JRC/GHSL/P2023A/GHS_POP")

In [ ]:
scale = 100

def doOneYear(pop_ic, year):
    print('\n\n{0}\n'.format(year))
    urbext = ee.FeatureCollection('projects/wri-datalab/SCL-Cities/urbanextents__bycountry_{0}'.format(year))
    urbext_data = geemap.ee_to_pandas(urbext)
    global totalua
    global totalpop
    urbarea_res = defaultdict(list)
    pop_res = defaultdict(list)
    print(year)
    for i in range(len(urbext_data)):
        ua = urbext_data.iloc[i]
        ua_f = urbext.filter(ee.Filter.eq('city_ids', str(ua['city_ids'])))
        geom = ua_f.geometry()
        if ua['country'] in popcodes:
            local_area = ee.Image.pixelArea().reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).get('area').getInfo()
            localpop_img = pop_ic.select('population_count').first()
            localpop = localpop_img.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).get('population_count').getInfo()
            urbarea_res[ua['country']].append(local_area)
            pop_res[ua['country']].append(localpop)
            totalua[year].append(local_area)
            totalpop[year].append(localpop)
        print(i, end=' ')
    return urbarea_res, pop_res

uares = {}
popres  = {}
totalua = {}
totalpop = {}
for year in [2000, 2005, 2010, 2015, 2020]:
    totalua[year] = []
    totalpop[year] = []
    uares[year], popres[year] = doOneYear(ghspop_ic.filter(ee.Filter.calendarRange(year, year, 'year')), year)

In [ ]:
YEARS = [2000, 2005, 2010, 2015, 2020]
bycountry = {}

for year in YEARS:
    if not 'global' in list(bycountry.keys()):
        bycountry['global'] = {}
    bycountry['global'] = bycountry['global'] | {
        'urban_area_sqmeters_{0}'.format(year): sum(totalua[year]),
        'urban_population_{0}'.format(year): sum(totalpop[year]),
        'urban_popdensity_persons-per-sqmeter_{0}'.format(year): sum(totalpop[year]) / sum(totalua[year])
    }

    country_list = list(uares[year].keys())
    country_list.sort()
    for country in country_list:
        if not country in list(bycountry.keys()):
            bycountry[country] = {}
        bycountry[country] = bycountry[country] | {
            'urban_area_sqmeters_{0}'.format(year): sum(uares[year][country]),
            'urban_population_{0}'.format(year): sum(popres[year][country]),
            'urban_popdensity_persons-per-sqmeter_{0}'.format(year): sum(popres[year][country]) / sum(uares[year][country])
        }

res_pd = pd.DataFrame.from_dict(bycountry, orient='index')
res_pd.to_csv('CTY-1_popdensity_v3a.csv')